In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!ls drive/My\ Drive/zindi

01.first_try_with_splitted_spec.ipynb
01.spectrograms_all_loop1.h5
01.spectrograms_all_loop2.h5
01.spectrograms_all_loop3.h5
01.spectrograms_first_try_batch_size_32_loop.h5
02_spec_all.ipynb
03_spectrograms_clean.ipynb
04_spectrograms_catboost_clean.ipynb
audio_signal_3d.jpeg
Graph
references
SampleSubmission.csv
Spectrograms_1_Train
Spectrograms_2_Test
Spectrograms_all_classified
Spectrograms_splitted_classified
submission_iteration_1.csv
submission_iteration_2.csv
submission_iteration_3.csv
Test
Test.csv
Train
Train.csv
viz.png
x_test.csv
x_train.csv


In [0]:
import pandas as pd
import numpy as np

import IPython.display as ipd
from matplotlib import pyplot as plt
import seaborn as sns

import librosa # package for music and audio processing, & features extraction 
import os, shutil, glob

set the path

In [0]:
path_colab = 'drive/My Drive/zindi/'
path_jupyt = './'

# set to True with colab or False with jupyter
colab = True
path = path_colab if colab else path_jupyt

## Take a look at the submission 

In [9]:
sub = pd.read_csv(path + 'SampleSubmission.csv')

# retrieve all the class names in a list (the 1st col is the id)
birds = sub.columns[1:]

# add a col with all files' paths 
sub['file_path'] = path + 'Test/' + sub['ID'] + '.mp3'
sub.head()

,ID,Ring-necked Dove,Black Cuckoo,Red-chested Cuckoo,Fiery-necked Nightjar,Green Wood Hoopoe,Crested Barbet,Cape Batis,Olive Bushshrike,Orange-breasted Bushshrike,Bokmakierie,Black-backed Puffback,Southern Boubou,Black-headed Oriole,Fork-tailed Drongo,African Paradise Flycatcher,Sabota Lark,Eastern Clapper Lark,Rufous-naped Lark,Dark-capped Bulbul,Sombre Greenbul,Long-billed Crombec,African Reed Warbler,Rattling Cisticola,Levaillants Cisticola,Tawny-flanked Prinia,Karoo Prinia,Bar-throated Apalis,Green-backed Camaroptera,Chestnut-vented Warbler,Cape White-eye,Karoo Scrub Robin,White-browed Scrub Robin,Cape Robin-Chat,White-browed Robin-Chat,Chorister Robin-Chat,Southern Double-collared Sunbird,White-bellied Sunbird,African Pipit,African Rock Pipit,Cape Bunting,file_path
0,019OYB,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,drive/My Drive/zindi/Test/019OYB.mp3
1,01S9OX,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,drive/My Drive/zindi/Test/01S9OX.mp3
2,02CS12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,drive/My Drive/zindi/Test/02CS12.mp3
3,02LM3W,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,drive/My Drive/zindi/Test/02LM3W.mp3
4,0C3A2V,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,drive/My Drive/zindi/Test/0C3A2V.mp3


List of species:

In [10]:
birds.values

array(['Ring-necked Dove', 'Black Cuckoo', 'Red-chested Cuckoo',
       'Fiery-necked Nightjar', 'Green Wood Hoopoe', 'Crested Barbet',
       'Cape Batis', 'Olive Bushshrike', 'Orange-breasted Bushshrike',
       'Bokmakierie', 'Black-backed Puffback', 'Southern Boubou',
       'Black-headed Oriole', 'Fork-tailed Drongo',
       'African Paradise Flycatcher', 'Sabota Lark',
       'Eastern Clapper Lark', 'Rufous-naped Lark', 'Dark-capped Bulbul',
       'Sombre Greenbul', 'Long-billed Crombec', 'African Reed Warbler',
       'Rattling Cisticola', 'Levaillants Cisticola',
       'Tawny-flanked Prinia', 'Karoo Prinia', 'Bar-throated Apalis',
       'Green-backed Camaroptera', 'Chestnut-vented Warbler',
       'Cape White-eye', 'Karoo Scrub Robin', 'White-browed Scrub Robin',
       'Cape Robin-Chat', 'White-browed Robin-Chat',
       'Chorister Robin-Chat', 'Southern Double-collared Sunbird',
       'White-bellied Sunbird', 'African Pipit', 'African Rock Pipit',
       'Cape Bunting'], 

In [11]:
# check if ID are alphabetically ordered
col_id = list(sub['ID'].values)
col_id == sorted(list(sub['ID'].values))

True

In [12]:
sub['len'] = sub['ID'].apply(lambda x: len(x))
sub['len'].value_counts()

6    911
Name: len, dtype: int64

In [13]:
nb_class = len(birds)
nb_class

40

## The train dataset & images

In [14]:
# same thing with train
train = pd.read_csv(path + 'Train.csv')
train['file_path'] = 'drive/My Drive/zindi/Spectrograms_1_Train/' + train['ID'] + '.png'
train.head()

,ID,common_name,file_path
0,MBMG2C,Ring-necked Dove,drive/My Drive/zindi/Spectrograms_1_Train/MBMG...
1,K8LJSB,Ring-necked Dove,drive/My Drive/zindi/Spectrograms_1_Train/K8LJ...
2,OGD9L6,Ring-necked Dove,drive/My Drive/zindi/Spectrograms_1_Train/OGD9...
3,581PCQ,Ring-necked Dove,drive/My Drive/zindi/Spectrograms_1_Train/581P...
4,P91M1F,Ring-necked Dove,drive/My Drive/zindi/Spectrograms_1_Train/P91M...


## Creation of images as numpy arrays

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

In [0]:
input_shape = (512, 512, 3)

In [0]:
def img_to_nparray(img_path):
    np_array = img_to_array(load_img(path=img_path))
    np_array = np_array.reshape(1, -1)
    return np_array[0] / 255

In [18]:
train.head()

,ID,common_name,file_path
0,MBMG2C,Ring-necked Dove,drive/My Drive/zindi/Spectrograms_1_Train/MBMG...
1,K8LJSB,Ring-necked Dove,drive/My Drive/zindi/Spectrograms_1_Train/K8LJ...
2,OGD9L6,Ring-necked Dove,drive/My Drive/zindi/Spectrograms_1_Train/OGD9...
3,581PCQ,Ring-necked Dove,drive/My Drive/zindi/Spectrograms_1_Train/581P...
4,P91M1F,Ring-necked Dove,drive/My Drive/zindi/Spectrograms_1_Train/P91M...


In [0]:
#type(train.drop(columns=['ID', 'common_name']).head())

In [0]:
#train.drop(columns=['ID', 'common_name']).apply(func=img_to_nparray, axis=1,  "result_type='expand')

In [21]:
train['file_path'].apply(img_to_nparray)

KeyboardInterrupt: ignored

In [0]:
#df.apply(lambda row: fn(row.text), axis='columns', result_type='expand')
train.apply(lambda row: img_to_nparray(row['file_path']), axis='columns', result_type='expand')

In [0]:
train.iloc[0, :]['file_path']

In [0]:
sdfsdf

In [22]:
print('test')

test


In [0]:
######### !!!!!!!!!!! change 20

def tf_predictions(model, csv_file, sub_df):
    probabilities = model.predict_generator(generator=test_generator, steps=911 // batch_size + 1)
    pred = pd.DataFrame(probabilities, columns=list(labels.values()))

    # Get filenames (set shuffle=false in generator is important)
    filenames = test_generator.filenames
    
    # remove path to keep img or ID name, add it to df
    pred['ID'] = [file_path[20:-4] for file_path in filenames]
    pred.head()
    
    
    # reorder predictions columns in the same way as submission
    try:
        sub_df_temp = sub_df.drop(columns=['len', 'file_path'])
    except:
        pass
    pred = pred[list(sub_df_temp.columns)]
    pred.head()

    
    # check if ID are alphabetically ordered then create the csv file for submission
    col_id = list(pred['ID'].values)
    if col_id == sorted(list(pred['ID'].values)):
        pred.to_csv(path + csv_file, index=False)
    else:
        print("ID aren't alphabetically ordered")

tf_predictions(model, 'submission_iteration_3.csv', sub)

submission scores for various iterations of 40 epochs : 
- after 1 iteration(s) on the training data set ie 040 epochs : accuracy  ~27% // submission : log loss on test dataset = 5.317
- after 2 iteration(s) on the training data set ie 080 epochs : accuracy  ~32% // submission : log loss on test dataset = 3.265
- after 3 iteration(s) on the training data set ie 120 epochs : accuracy  ~43% // submission : log loss on test dataset = 8.137
